<a href="https://colab.research.google.com/github/ShussainML/500-AI-Machine-learning-Deep-learning-Computer-vision-NLP-Projects-with-code/blob/main/nlp_project_model_training_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch torchtext spacy transformers
!python -m spacy download en_core_web_sm
!pip install streamlit

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [12]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
import spacy
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
from sklearn.metrics import f1_score
from torch.utils.tensorboard import SummaryWriter
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')
lemmatizer = WordNetLemmatizer()

# Load the CSV file
try:
    train_dataframe = pd.read_csv('/content/mega_train.csv')
except pd.errors.ParserError:
    print("There is an issue with your CSV file format.")

try:
    test_dataframe = pd.read_csv('/content/mega_test.csv')
except pd.errors.ParserError:
    print("There is an issue with your CSV file format.")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


There is an issue with your CSV file format.
There is an issue with your CSV file format.


In [13]:
# Preprocessing function
def preprocess_text(text):
    doc = nlp(text)
    tokens = [lemmatizer.lemmatize(token.text.lower()) for token in doc if token.is_alpha]
    return ' '.join(tokens)

train_dataframe['text'] = train_dataframe['text'].apply(preprocess_text)
test_dataframe['text'] = test_dataframe['text'].apply(preprocess_text)

# Create new col with author number mappings for ML
auth_sort = sorted(train_dataframe['Author'].unique())
dictOfAuthors = {i: auth_sort[i] for i in range(len(auth_sort))}
swap_dict = {value: key for key, value in dictOfAuthors.items()}
train_dataframe['Author_num'] = train_dataframe['Author'].map(swap_dict)
test_dataframe['Author_num'] = test_dataframe['Author'].map(swap_dict)

# Drop unnecessary columns
train_dataframe = train_dataframe.drop(columns=['Author', 'index'])
test_dataframe = test_dataframe.drop(columns=['Author', 'index', 'hopeful_test'])

# Tokenization and padding
max_length = 100
tokenizer = nltk.word_tokenize
def tokenize_and_pad(texts, max_length):
    tokenized_texts = [tokenizer(text) for text in texts]
    padded_texts = [text[:max_length] + ['<PAD>'] * (max_length - len(text)) if len(text) < max_length else text[:max_length] for text in tokenized_texts]
    return padded_texts

train_texts = tokenize_and_pad(train_dataframe['text'].tolist(), max_length)
test_texts = tokenize_and_pad(test_dataframe['text'].tolist(), max_length)

# Build vocabulary
all_tokens = [token for text in train_texts + test_texts for token in text]
vocab = {token: idx for idx, token in enumerate(set(all_tokens))}
vocab['<PAD>'] = 0

def texts_to_sequences(texts, vocab):
    return [[vocab[token] for token in text] for text in texts]

train_sequences = texts_to_sequences(train_texts, vocab)
test_sequences = texts_to_sequences(test_texts, vocab)

train_inputs = torch.tensor(train_sequences)
test_inputs = torch.tensor(test_sequences)

train_labels = torch.tensor(train_dataframe['Author_num'].values)
test_labels = torch.tensor(test_dataframe['Author_num'].values)

# Create DataLoader
train_data = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

test_data = TensorDataset(test_inputs, test_labels)
test_loader = DataLoader(test_data, batch_size=16)


NameError: name 'train_dataframe' is not defined

In [5]:
# Model definition
class AuthorClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size, num_layers=1, model_type='lstm'):
        super(AuthorClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.model_type = model_type
        if model_type == 'lstm':
            self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        elif model_type == 'gru':
            self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        if self.model_type == 'lstm':
            _, (hidden, _) = self.rnn(embedded)
        elif self.model_type == 'gru':
            _, hidden = self.rnn(embedded)
        hidden = hidden[-1]
        output = self.fc(hidden)
        return output



In [6]:
# Training and evaluation functions
def train_classifier(model, dataset_loader, loss_function, optimizer, num_epochs, log="runs", verbose=False):
    writer = SummaryWriter(log_dir=log)
    model.train()
    step = 0
    f1score_train = []
    accuracy_train = []
    loss_train = []

    for epoch in range(num_epochs):
        correct = 0
        total = 0
        total_loss = 0
        f1 = 0
        f1_step = 0

        for batch in dataset_loader:
            text, labels = batch
            text, labels = text.to(device), labels.to(device)

            optimizer.zero_grad()
            output = model(text)

            loss = loss_function(output, labels)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()

            pred = torch.max(output.data, 1).indices
            f1 += f1_score(labels.cpu().numpy(), pred.cpu().numpy(), average='macro')
            correct += (pred == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()
            f1_step += 1

            if step % 100 == 0:
                writer.add_scalar("Loss/train", total_loss/total, step)
                writer.add_scalar("Acc/train", correct/total, step)
                writer.add_scalar("F1 Score/train", f1/f1_step, step)

            step += 1

        f1score_train.append(f1/f1_step)
        loss_train.append(total_loss/total)
        accuracy_train.append(correct/total)
        print(f'Epoch: {epoch+1}, Acc: {correct/total}, Loss: {total_loss/total}, F1 Score: {f1/f1_step}')

    return loss_train, f1score_train, accuracy_train




In [7]:
def evaluate_classifier(model, dataset_loader, loss_function):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0
    f1 = 0
    f1_step = 0

    overall_pred = []
    overall_label = []

    with torch.no_grad():
        for batch in dataset_loader:
            text, labels = batch
            text, labels = text.to(device), labels.to(device)
            output = model(text)
            loss = loss_function(output, labels)
            pred = torch.max(output.data, 1).indices

            correct += (pred == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()
            f1 += f1_score(labels.cpu().numpy(), pred.cpu().numpy(), average='macro')
            f1_step += 1

            overall_pred.extend(pred.cpu().numpy())
            overall_label.extend(labels.cpu().numpy())

    f1_score_val = f1 / f1_step
    accuracy_val = correct / total
    loss_val = total_loss / total

    print(f'Validation - Acc: {accuracy_val}, Loss: {loss_val}, F1 Score: {f1_score_val}')
    return overall_pred, overall_label, accuracy_val, f1_score_val, loss_val




In [8]:
# Training the Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

output_size = len(auth_sort)
hidden_size = 300
vocab_size = len(vocab)
embedding_length = 100
num_epochs = 1
model_type = 'lstm'  # or 'gru'

model = AuthorClassifier(vocab_size, embedding_length, hidden_size, output_size, model_type=model_type)
model = model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
log_dir = f'runs/{model_type}1'

final_acc_train = []
final_loss_train = []
final_loss_test = []
final_acc_test = []
final_f1score_train = []
final_f1score_test = []

for _ in range(20):
    loss_train, f1score, accs = train_classifier(model, train_loader, loss_function, optimizer, num_epochs, log=log_dir)
    overall_pred, overall_label, accs_test, f1ss, loss_test = evaluate_classifier(model, test_loader, loss_function)
    final_acc_train.append(accs[0])
    final_acc_test.append(accs_test)
    final_f1score_train.append(f1score[0])
    final_f1score_test.append(f1ss)
    final_loss_train.append(loss_train[0])
    final_loss_test.append(loss_test)


# Save the model
torch.save(model.state_dict(), 'author_classifier_model.pth')
# Plotting confusion matrix
cf = confusion_matrix(overall_label, overall_pred)
plt.figure(figsize=(10,10))
ax = sns.heatmap(cf, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

NameError: name 'auth_sort' is not defined

In [1]:
# Streamlit App
import streamlit as st

# Function to preprocess and predict
def preprocess_and_predict(text):
    text = preprocess_text(text)
    tokenized_text = tokenize_and_pad([text], max_length)
    sequence = texts_to_sequences(tokenized_text, vocab)
    input_tensor = torch.tensor(sequence).to(device)
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
    pred = torch.argmax(output, dim=1).item()
    return dictOfAuthors[pred]

# Streamlit UI
st.title('Author Prediction')
user_input = st.text_area("Enter Text Here")
if st.button("Predict"):
    prediction = preprocess_and_predict(user_input)
    st.write(f'This text is likely written by {prediction}')

ModuleNotFoundError: No module named 'streamlit'

In [ ]:
# Save the model
torch.save(model.state_dict(), 'author_classifier_model.pth')


In [ ]:
# Create the Streamlit app script
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

class AuthorClassifier(nn.Module):
    def __init__(self, mode, output_size, hidden_size, vocab_size, embedding_length):
        super(AuthorClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(768, output_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs[1]  # [CLS] token
        output = self.fc(cls_output)
        return output

model = AuthorClassifier(mode='lstm', output_size=50, hidden_size=300, vocab_size=30522, embedding_length=100)
model.load_state_dict(torch.load('author_classifier_model.pth', map_location=torch.device('cpu')))
model.eval()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_and_predict(text):
    inputs = tokenizer(text, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    with torch.no_grad():
        output = model(input_ids, attention_mask)
        predicted_class = torch.argmax(output, dim=1).item()
    return predicted_class

st.title("Author Classifier")
st.write("Enter some text and the model will predict the author.")

user_input = st.text_area("Text input")
if st.button("Predict"):
    prediction = preprocess_and_predict(user_input)
    st.write(f"The predicted author class is: {prediction}")





Overwriting app.py
